In [26]:
import numpy as np
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from astropy.io import fits
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

In [24]:
pgcc_data=fits.open('/Users/codydirks/PGCC/HFI_PCCS_GCC_R2.02.fits')[1].data

# Only selects PGCCs with a known distance (ignores those with just upper limits)
pgcc_data=pgcc_data[np.where(pgcc_data['dist']>0)]
len(pgcc_data)

4964

In [27]:
# Selects all Gaia targets in a circle centered on each PGCC
# Circle size is given by the GAU_MAJOR_AXIS of the PGCC fit,
# thus over selecting targets

pgcc_x_gaia=[]
for pgcc in pgcc_data[0:2]:
    query=' '.join([
                'SELECT *',
               'FROM gaiadr2.gaia_source',
               'WHERE 1=CONTAINS(POINT(\'ICRS\', ra, dec),',
                                'CIRCLE(\'ICRS\', {0},{1},{2}))'.format(pgcc['ra'],pgcc['dec'],pgcc['gau_major_axis']/60.)
                ])
    results=Gaia.launch_job_async(query).get_results()
    pgcc_x_gaia.append((pgcc,results))
    clear_output()

In [28]:
for result in pgcc_x_gaia:
    pgcc=result[0]
    gaia=result[1]
    pgcc_dist=round(1000.*pgcc['dist'],2)
    i=0
    for row in gaia:
        if 1000./row['parallax'] > pgcc_dist:
            i=i+1
    print i

211
98
